# STEP 1: Importing libraries and data

In [9]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [10]:
pwd = os.getcwd()

train = pd.read_csv(pwd+'/train.csv')
test = pd.read_csv(pwd+'/test.csv')

In [11]:
X_train = train.copy()
y_train = pd.DataFrame(X_train.pop('label'))
X_test = test.copy()

# STEP 2: Feature Engineering

In [12]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [13]:
X_train = (np.array(X_train)).reshape(-1,28,28,1)
X_test = (np.array(X_test)).reshape(-1,28,28,1)

In [14]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes = 10)



# STEP 3: Modeling

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Input, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

 
imgdatagenerator = ImageDataGenerator(
    rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    zoom_range=0.1,  
    shear_range=0.1,  
    horizontal_flip=False
)

imgdatagenerator.fit(X_train)


early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=3,  
    restore_best_weights=True,
    verbose=0
)

1st CNN model

In [ ]:
cnn1_filter1 = [32,64]
cnn1_filter2 = [64,128]
cnn1_dropout = [0.3,0.5]

best_cnn1_filter1 = 0
best_cnn1_filter2 = 0
best_cnn1_dropout = 0.0
best_cnn1_accuracy = 0

for f1 in cnn1_filter1:
    for f2 in cnn1_filter2:
        for d in cnn1_dropout:

            cnn1 = Sequential([

            Conv2D(f1,(3,3),activation='relu',input_shape=(28,28,1)),
            BatchNormalization(),
            MaxPooling2D((2,2)),


            Conv2D(f2,(3,3),activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Flatten(),
            Dense(128, activation='relu'),
            Dropout(d),
            Dense(10, activation='softmax')])



            cnn1.compile(optimizer='adam', 
             loss='categorical_crossentropy', 
             metrics=['accuracy'])
            

            model = cnn1.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                            epochs=15,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping],
                            verbose=0)
            
            val_accuracy = max(model.history['val_accuracy'])

            if val_accuracy > best_cnn1_accuracy:
                best_cnn1_accuracy = val_accuracy
                best_cnn1_dropout = d
                best_cnn1_filter1 = f1
                best_cnn1_filter2 = f2
            





2nd CNN Model

In [ ]:
cnn2_filter = [16,32]
cnn2_dropout = [0.4,0.6]

best_cnn2_dropout = 0.0
best_cnn2_filter = 0
best_cnn2_accuracy = 0

for f in cnn2_filter:
    for d in cnn2_dropout:

        cnn2 = Sequential([

            Conv2D(f,(3,3),activation='relu',input_shape=(28,28,1),padding='same'),
            BatchNormalization(),

            Conv2D(f,(3,3),activation='relu',padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Conv2D(f*2,(3,3),activation='relu',padding='same'),
            BatchNormalization(),

            Conv2D(f*2,(3,3),activation='relu',padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Flatten(),
            Dense(64, activation='relu'),
            Dropout(d),
            Dense(10, activation='softmax')

        ])

        cnn2.compile(optimizer='adam', 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])
        
        model = cnn2.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                            epochs=15,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping],
                            verbose=0)
        
        val_accuracy = max(model.history['val_accuracy'])

        if val_accuracy > best_cnn2_accuracy:
            best_cnn2_accuracy = val_accuracy
            best_cnn2_dropout = d
            best_cnn2_filter = f
            
        


3rd CNN Model

In [ ]:
cnn3_filter = [32,64]
cnn3_dropout = [0.3,0.4]

best_cnn3_filter = 0
best_cnn3_dropout = 0.0
best_cnn3_accuracy = 0

for f in cnn3_filter:
    for d in cnn3_dropout:


        cnn3 = Sequential([

            Conv2D(f,(5,5),activation='relu',input_shape=(28,28,1),padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Conv2D(f*2, (3,3), activation='relu', padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),


            Conv2D(f*3, (3,3), activation='relu', padding='same'),
            BatchNormalization(),


            Flatten(),
            Dense(96, activation='relu'),
            Dropout(d),
            Dense(10, activation='softmax')
        ])


        cnn3.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
        

        model = cnn3.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                        epochs=15,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping],
                        verbose=0)
        
        val_accuracy = max(model.history['val_accuracy'])

        if val_accuracy > best_cnn3_accuracy:
            best_cnn3_accuracy = val_accuracy
            best_cnn3_dropout = d
            best_cnn3_filter = f

In [ ]:
cnn1_final = Sequential([

            Conv2D(best_cnn1_filter1,(3,3),activation='relu',input_shape=(28,28,1)),
            BatchNormalization(),
            MaxPooling2D((2,2)),


            Conv2D(best_cnn1_filter2,(3,3),activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Flatten(),
            Dense(128, activation='relu'),
            Dropout(best_cnn1_dropout),
            Dense(10, activation='softmax')])



cnn1_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


cnn1_final.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                epochs=15,
                validation_data=(X_val, y_val),
                callbacks=[early_stopping],
                verbose=0)

In [ ]:
cnn2_final = Sequential([

            Conv2D(best_cnn2_filter,(3,3),activation='relu',input_shape=(28,28,1),padding='same'),
            BatchNormalization(),

            Conv2D(best_cnn2_filter,(3,3),activation='relu',padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Conv2D(best_cnn2_filter*2,(3,3),activation='relu',padding='same'),
            BatchNormalization(),

            Conv2D(best_cnn2_filter*2,(3,3),activation='relu',padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Flatten(),
            Dense(64, activation='relu'),
            Dropout(best_cnn2_dropout),
            Dense(10, activation='softmax')

        ])

cnn2_final.compile(optimizer='adam', 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])
        

cnn2_final.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                epochs=15,
                validation_data=(X_val, y_val),
                callbacks=[early_stopping],
                verbose=0)

In [ ]:
cnn3_final = Sequential([

            Conv2D(best_cnn3_filter,(5,5),activation='relu',input_shape=(28,28,1),padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),

            Conv2D(best_cnn3_filter*2, (3,3), activation='relu', padding='same'),
            BatchNormalization(),
            MaxPooling2D((2,2)),


            Conv2D(best_cnn3_filter*3, (3,3), activation='relu', padding='same'),
            BatchNormalization(),


            Flatten(),
            Dense(96, activation='relu'),
            Dropout(best_cnn3_dropout),
            Dense(10, activation='softmax')
        ])


cnn3_final.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
        

cnn3_final.fit(imgdatagenerator.flow(X_train, y_train, batch_size=128),
                        epochs=15,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping],
                        verbose=0)

Integrated result

In [ ]:
final_pred = []

pred1 = cnn1_final.predict(X_test)
pred2 = cnn2_final.predict(X_test)
pred3 = cnn3_final.predict(X_test)

for i in range(len(X_test)):
    
    pred1_num = np.argmax(pred1[i])
    pred2_num = np.argmax(pred2[i])
    pred3_num = np.argmax(pred3[i])

    if pred1_num!=pred2_num and pred2_num!=pred3_num and pred1_num!=pred3_num:
        prob1 = pred1[i][pred1_num]  
        prob2 = pred2[i][pred2_num]  
        prob3 = pred3[i][pred3_num]
        
        max_prob_index = np.argmax([prob1,prob2,prob3])
        final_pred.append([pred1_num, pred2_num, pred3_num][max_prob_index])

    else:
        counts = np.bincount([pred1_num,pred2_num,pred3_num])
        most_frequent_value = np.argmax(counts)
        final_pred.append(most_frequent_value)




In [ ]:
submission = pd.DataFrame({'ImageId': range(1, len(X_test)+1),
                            'Label': final_pred})


submission.to_csv('submission.csv', index=False)